# THE "HELLO WORLD" OF MACHINE LEARNING - MNIST CLASSIFICATION

## Training the Model

### 1: Import Required Module(s)

In [ ]:
# TensorFlow is the machine learning framework. It is built off of the Keras API.
import tensorflow as tf

### 2: Load the Images and Labels
For now, we only need the training dataset. \
It is crucial that the training dataset and the testing dataset are never combined in any capacity.

In [ ]:
# The images are what is fed into the model as input. All images are just arrays of pixel values (e.g., RGBA).
# The labels correspond to the digit 0-9 represented in each image. These are used to both train the model and compare its answers.
(training_images, training_labels), _ = tf.keras.datasets.mnist.load_data()

### 3: Data Preprocessing

In [ ]:
# Cast the images from using unsigned 8-bit integers to 32-bit floats, then normalize.
# This step alone increases the final accuracy of the model by approximately 5% of the dataset.
training_images = training_images.astype('float32') / 255

### 4: Create the Model

In [ ]:
# Since we are building a fairly straightforward model, we can use the Sequential API provided by Keras.
# For more complex models, use the Functional API.
model = tf.keras.models.Sequential([
    # The input layer is of the same dimension as the images. MNIST uses 28x28 pixel images.
    tf.keras.layers.Flatten(input_shape=(28,28)),
    # The hidden layer(s), or "meat" of the neural network. In general, more hidden layers means a more capable or complex network.
    tf.keras.layers.Dense(128, activation='relu'),
    # For classification, the number of nodes in the output layer corresponds to the number of categories, or in this case, digits.
    tf.keras.layers.Dense(10)                      
])

### 5: Compile the Model

In [ ]:
# Build the model with the selected optimizer, loss function, and metrics to analyze during training.
model.compile(
    # Determines how model weights are updated in response to the loss function.
    optimizer=tf.keras.optimizers.Adam(0.001),                            
    # Determines how harshly to penalize incorrect model predictions.
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    # Metrics do not update model weights. They are only used for callbacks and sanity checks.
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],               
)

### 6: Train the Model

In [ ]:
# Specify input and target values to the model, along with parameters such as batch size.
history = model.fit(
    training_images,        # The images for the model to train on.
    training_labels,        # The correct labels for the training images.
    batch_size=2**7,        # Number of samples before the model updates its parameters.
    epochs=6,               # How many times the model runs over a dataset.
    validation_split=0.20,  # The percentage of training data to be used as validation data.
    shuffle=True            # Boolean indicating whether the input should be shuffled between each epoch.
)

## Evaluating the Model

### 1: Import Required Module(s)

In [ ]:
# NumPy is an advanced, highly optimized mathematical library
import numpy as np
# Matplotlib is a plotting module that allows us to easily visualize data.
from matplotlib import pyplot as plt

### 2: Load the Images and Labels
Now, we need the testing dataset.

In [ ]:
# In testing, only the images are provided to the model, not the labels.
# The labels now are for the user to compare with the models' predictions.
_, (testing_images, testing_labels) = tf.keras.datasets.mnist.load_data()

### 3: Data Preprocessing (notice a pattern?)
It is crucial that the testing data is processed exactly the same as the training data. \
Otherwise, the results of the model will be misleading.

In [ ]:
# Once again, cast the images from using unsigned 8-bit integers to 32-bit floats, then normalize.
testing_images = testing_images.astype('float32') / 255

### 4: Predict Testing Data

In [ ]:
# As mentioned previously, only the testing images are provided to the model for predictions.
# Providing the labels would be cheating! Providing training data would also be cheating, since the model has already seen those images.
predictions = model.predict(testing_images)

### 5: Process Predictions

In [ ]:
# The format of the predictions output matches the format of the output of the model.
# Since the model has 10 output nodes, the result of each prediction is a list of 10 elements.
# The way to intepret this prediction list is that each element is a likelihood value corresponding to each possible output category, or digit.
# The index of each likelihood value in the list corresponds to the digit whose likelihood it is representing.
# High-magnitude likelihood values correspond to strong model confidence in that particular category.
# Positive likelihood values indicate the model "agrees" with a particular digit, and vice-versa.
# The digit that the model predicts for each image is the one with the most positive likelihood value.
discretized_predictions = np.argmax(predictions, axis=1)

### 6: Analyze Results
We have arrived to the final step in the machine learning process - assessing model performance on unseen data. \
This is mostly a sandbox at this point where we try to look for trends in the output.

#### Total Accuracy

In [ ]:
num_predictions = len(discretized_predictions)
num_successful_predictions = sum(discretized_predictions == testing_labels)

print(f'Total Accuracy:  {100 * num_successful_predictions / num_predictions:.2f}%')

#### Category Specific Accuracy

In [ ]:
for digit in range(10):
    digit_specific_image_ids = testing_labels == digit
    num_digit_specific_predictions = sum(digit_specific_image_ids)
    num_successful_digit_specific_predictions = sum(discretized_predictions[digit_specific_image_ids] == testing_labels[digit_specific_image_ids])

    print(f'Accuracy for {digit}:  {100 * num_successful_digit_specific_predictions / num_digit_specific_predictions:.2f}%')

#### Correct Predictions

In [ ]:
correct_prediction_image_ids = np.nonzero(discretized_predictions == testing_labels)[0]

random_correct_image_id = np.random.choice(correct_prediction_image_ids)

plt.axis('off')
plt.title(f'Prediction: {discretized_predictions[random_correct_image_id]}        True Value: {testing_labels[random_correct_image_id]}')
plt.imshow(testing_images[random_correct_image_id].reshape(28,28), cmap='gray')
plt.show()

#### Identifying Problematic Inputs

In [ ]:
incorrect_prediction_image_ids = np.nonzero(discretized_predictions != testing_labels)[0]

print(incorrect_prediction_image_ids)

#### How Much Better Can WE Do?

In [ ]:
random_incorrect_image_id = np.random.choice(incorrect_prediction_image_ids)

plt.axis('off')
plt.title(f'Prediction: {discretized_predictions[random_incorrect_image_id]}        True Value: {testing_labels[random_incorrect_image_id]}')
plt.imshow(testing_images[random_incorrect_image_id].reshape(28,28), cmap='gray')
plt.show()

# Interactive Website:
# playground.tensorflow.org